In [1]:
from pyspark.context import SparkContext,SparkConf
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [2]:

T= sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/temperature-readings-tiny.csv")

### Assignments

In [3]:
lines = T.map(lambda line: line.split(";"))

year_temperature = lines.map(lambda x: (x[1][0:4], float(x[3])))

year_temperature = year_temperature.filter(lambda x: int(x[0]) >= 1950 and int(x[0]) <= 2014)

max_temperatures= year_temperature.reduceByKey(lambda x,y :x if x>=y else y)

max_temperaturesSorted=max_temperatures.sortBy(ascending=False,keyfunc=lambda k:k[1])

min_temperatures= year_temperature.reduceByKey(lambda x,y :x if x<=y else y)

min_temperaturesSorted=min_temperatures.sortBy(ascending=True,keyfunc=lambda k:k[1])


In [4]:
print("----Table with max temperatures per year---")
max_temperaturesSorted.take(10)


----Table with max temperatures per year---


[('2014', 29.1),
 ('1960', 29.0),
 ('1959', 28.2),
 ('1958', 28.1),
 ('1956', 26.0),
 ('1957', 25.2),
 ('1955', 20.4),
 ('1961', 19.0),
 ('2013', 10.2)]

In [5]:
print("----Table with min temperatures per year---")
min_temperaturesSorted.take(10)


----Table with min temperatures per year---


[('1956', -30.0),
 ('1960', -28.3),
 ('1958', -27.9),
 ('1955', -26.2),
 ('2014', -24.3),
 ('1961', -23.5),
 ('1959', -23.2),
 ('1957', -19.9),
 ('2013', -13.3)]

### Question 1.a

In [6]:
year_temp = lines.map(lambda x: (x[1][0:4],( float(x[3]),x[0])))

year_temp = year_temp.filter(lambda x: int(x[0]) >= 1950 and int(x[0]) <= 2014)

#############################
max_temperatures1= year_temp.reduceByKey(lambda x,y :x if x>=y else y)

max_temperaturesSorted1=max_temperatures1.sortBy(ascending=False,keyfunc=lambda k:k[1][0])

t=max_temperaturesSorted1.map(lambda x: (x[0], float(x[1][0]),x[1][1]))

#############################

min_temperatures1= year_temp.reduceByKey(lambda x,y :x if x<=y else y)

min_temperaturesSorted1=min_temperatures1.sortBy(ascending=True,keyfunc=lambda k:k[1][0])

t1=min_temperaturesSorted1.map(lambda x: (x[0], float(x[1][0]),x[1][1]))


In [7]:
print("----Table with max temperatures per year---")
t.take(10)

----Table with max temperatures per year---


[('2014', 29.1, '102170'),
 ('1960', 29.0, '102190'),
 ('1959', 28.2, '102190'),
 ('1958', 28.1, '102190'),
 ('1956', 26.0, '102190'),
 ('1957', 25.2, '102190'),
 ('1955', 20.4, '102190'),
 ('1961', 19.0, '102190'),
 ('2013', 10.2, '102170')]

In [8]:
print("----Table with min temperatures per year---")
t1.take(10)

----Table with min temperatures per year---


[('1956', -30.0, '102190'),
 ('1960', -28.3, '102190'),
 ('1958', -27.9, '102190'),
 ('1955', -26.2, '102190'),
 ('2014', -24.3, '102170'),
 ('1961', -23.5, '102190'),
 ('1959', -23.2, '102190'),
 ('1957', -19.9, '102190'),
 ('2013', -13.3, '102170')]

### Question 1.b

In [9]:
import pandas as pd
dat=pd.read_csv("temperature-readings-tiny.csv",sep=';',header=None)
dat.columns = ['station_number', 'date','time','Temperature','quality']
print("first 5 rows")
dat.head(5)
# print("column names")
# dat.columns

first 5 rows


,station_number,date,time,Temperature,quality
0,102170,2013-11-01,06:00:00,6.8,G
1,102170,2013-11-01,18:00:00,3.8,G
2,102170,2013-11-02,06:00:00,5.8,G
3,102170,2013-11-02,18:00:00,-1.1,G
4,102170,2013-11-03,06:00:00,-0.2,G


In [10]:
dt=dat[['station_number','date','Temperature']]
dt['year'] = pd.DatetimeIndex(dt['date']).year
dt.head()


C:\Users\quartermaine\Anaconda2\envs\python3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,station_number,date,Temperature,year
0,102170,2013-11-01,6.8,2013
1,102170,2013-11-01,3.8,2013
2,102170,2013-11-02,5.8,2013
3,102170,2013-11-02,-1.1,2013
4,102170,2013-11-03,-0.2,2013


In [11]:
DT=dt[['station_number','year','Temperature']]
DT.head()

,station_number,year,Temperature
0,102170,2013,6.8
1,102170,2013,3.8
2,102170,2013,5.8
3,102170,2013,-1.1
4,102170,2013,-0.2


In [12]:
DT.groupby(['year','station_number']).agg({'Temperature': ['min', 'max']})

Temperature      
                            min   max
year station_number                  
1955 102190               -26.2  20.4
1956 102190               -30.0  26.0
1957 102190               -19.9  25.2
1958 102190               -27.9  28.1
1959 102190               -23.2  28.2
1960 102190               -28.3  29.0
1961 102190               -23.5  19.0
2013 102170               -13.3  10.2
2014 102170               -24.3  29.1
2015 102170               -19.6  26.8
2016 102170               -25.5  25.8

### Question 2

In [13]:
y = lines.map(lambda x: ( (x[1][0:4],x[1][5:7]),float(x[3]) ))

y = y.filter(lambda x: int(x[0][0]) >= 1950 and int(x[0][0]) <= 2014)

#############################
tempsOver= y.filter(lambda x : x[1] >= 10)

tempsOver_unpack=tempsOver.map(lambda x: ( (x[0][0], x[0][1]),1))

tempsOver10=tempsOver_unpack.reduceByKey(lambda x,y : x+y).sortByKey() #x,y are the value of each key ,addeed up
tempsOver10.take(20)


[(('1955', '09'), 55),
 (('1955', '10'), 23),
 (('1956', '03'), 4),
 (('1956', '04'), 7),
 (('1956', '05'), 60),
 (('1956', '06'), 92),
 (('1956', '07'), 108),
 (('1956', '08'), 84),
 (('1956', '09'), 54),
 (('1956', '10'), 17),
 (('1957', '03'), 1),
 (('1957', '04'), 13),
 (('1957', '05'), 46),
 (('1957', '06'), 72),
 (('1957', '07'), 109),
 (('1957', '08'), 96),
 (('1957', '09'), 43),
 (('1957', '10'), 9),
 (('1958', '04'), 4),
 (('1958', '05'), 40)]

In [14]:
# with one line 
sorted(tempsOver.countByKey().items())[0:10]

[(('1955', '09'), 55),
 (('1955', '10'), 23),
 (('1956', '03'), 4),
 (('1956', '04'), 7),
 (('1956', '05'), 60),
 (('1956', '06'), 92),
 (('1956', '07'), 108),
 (('1956', '08'), 84),
 (('1956', '09'), 54),
 (('1956', '10'), 17)]

### Distinct

In [15]:
help(tempsOver.distinct)

Help on method distinct in module pyspark.rdd:

distinct(numPartitions=None) method of pyspark.rdd.PipelinedRDD instance
    Return a new RDD containing the distinct elements in this RDD.
    
    >>> sorted(sc.parallelize([1, 1, 2, 3]).distinct().collect())
    [1, 2, 3]



In [16]:
# with one line
sorted(tempsOver.distinct().countByKey().items())[0:10]

[(('1955', '09'), 35),
 (('1955', '10'), 20),
 (('1956', '03'), 4),
 (('1956', '04'), 5),
 (('1956', '05'), 44),
 (('1956', '06'), 58),
 (('1956', '07'), 76),
 (('1956', '08'), 49),
 (('1956', '09'), 30),
 (('1956', '10'), 13)]

In [46]:
# test the influence of distinct 
# j = lines.map(lambda x: ( (x[1][0:4],x[1][5:7],x[1]),float(x[3]) ))

# j = j.filter(lambda x: int(x[0][0]) == 1956 and int(x[0][1])==4 and x[1]>=10.)

# j.take(10)

### Question 3

In [17]:
rdd = lines.map(lambda x: ( (x[1][0:4],x[1][5:7],x[1][8:10],x[0]),float(x[3]) ))

rdd = rdd.filter(lambda x: int(x[0][0]) >= 1960 and int(x[0][0]) <= 2014)

#############################

rdd=rdd.map(lambda x: ( (x[0][0],x[0][1],x[0][2], x[0][3]),float(x[1]) ) )

rdd.take(10)

[(('2013', '11', '01', '102170'), 6.8),
 (('2013', '11', '01', '102170'), 3.8),
 (('2013', '11', '02', '102170'), 5.8),
 (('2013', '11', '02', '102170'), -1.1),
 (('2013', '11', '03', '102170'), -0.2),
 (('2013', '11', '03', '102170'), 5.6),
 (('2013', '11', '04', '102170'), 6.5),
 (('2013', '11', '04', '102170'), 5.1),
 (('2013', '11', '05', '102170'), 4.2),
 (('2013', '11', '05', '102170'), 3.2)]

In [18]:
meanRDD = (rdd
           .mapValues(lambda x: (x, 1))
           .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))
           .mapValues(lambda x: x[0]/x[1]))
meanRDD.sortByKey(ascending=False).take(31)

[(('2014', '12', '31', '102170'), -7.05),
 (('2014', '12', '30', '102170'), -9.600000000000001),
 (('2014', '12', '29', '102170'), -12.8),
 (('2014', '12', '28', '102170'), -20.200000000000003),
 (('2014', '12', '27', '102170'), -21.05),
 (('2014', '12', '26', '102170'), -19.9),
 (('2014', '12', '25', '102170'), -18.65),
 (('2014', '12', '24', '102170'), -12.8),
 (('2014', '12', '23', '102170'), -11.850000000000001),
 (('2014', '12', '22', '102170'), -6.0),
 (('2014', '12', '21', '102170'), -10.0),
 (('2014', '12', '20', '102170'), -7.050000000000001),
 (('2014', '12', '19', '102170'), 0.20000000000000007),
 (('2014', '12', '18', '102170'), -3.2),
 (('2014', '12', '17', '102170'), -6.75),
 (('2014', '12', '16', '102170'), -0.45),
 (('2014', '12', '15', '102170'), 2.8),
 (('2014', '12', '14', '102170'), -1.2000000000000002),
 (('2014', '12', '13', '102170'), -1.85),
 (('2014', '12', '12', '102170'), 1.7000000000000002),
 (('2014', '12', '11', '102170'), 1.5),
 (('2014', '12', '10', '102

In [19]:
meanRDD.map(lambda x: ( (x[0][0],x[0][1],x[0][3]),x[1]) )\
.mapValues(lambda x: (x, 1))\
.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))\
.mapValues(lambda x: x[0]/x[1]).sortByKey(ascending=False).take(50)

[(('2014', '12', '102170'), -5.801612903225807),
 (('2014', '11', '102170'), 2.525),
 (('2014', '10', '102170'), 7.106451612903226),
 (('2014', '09', '102170'), 8.585),
 (('2014', '08', '102170'), 13.869354838709675),
 (('2014', '07', '102170'), 19.659677419354836),
 (('2014', '06', '102170'), 14.443333333333333),
 (('2014', '05', '102170'), 10.756451612903227),
 (('2014', '04', '102170'), 4.776666666666667),
 (('2014', '03', '102170'), 1.896774193548387),
 (('2014', '02', '102170'), 0.6749999999999999),
 (('2014', '01', '102170'), -4.106451612903226),
 (('2013', '12', '102170'), 0.7096774193548387),
 (('2013', '11', '102170'), -0.05166666666666663),
 (('1961', '05', '102190'), 10.61533333333333),
 (('1961', '04', '102190'), 4.507777777777778),
 (('1961', '03', '102190'), 2.708602150537634),
 (('1961', '02', '102190'), -2.1333333333333333),
 (('1961', '01', '102190'), -7.215053763440859),
 (('1960', '12', '102190'), -2.427419354838709),
 (('1960', '11', '102190'), -0.42999999999999994)

### Question 4


In [20]:
l = T.map(lambda line: line.split(";"))

station_temperature = l.map(lambda x:( (x[0],x[1]) ,float(x[3])))

#station_temperature = station_temperature.filter(lambda x: int(x[0]) >= 1950 and int(x[0]) <= 2014)

max_temp_station= station_temperature.reduceByKey(lambda x,y :x if x>=y else y)

max_temp_station=max_temp_station.filter(lambda x: x[1]>=25. and x[1]<=30.)

max_temp_st=max_temp_station.map(lambda x: (x[0],float(x[1]) ))
max_temp_st.take(5)

[(('102170', '2014-07-22'), 28.0),
 (('102170', '2014-07-23'), 29.1),
 (('102170', '2014-07-26'), 28.7),
 (('102170', '2014-07-29'), 25.5),
 (('102170', '2015-07-01'), 26.8)]

In [81]:
#station_temperature.take(10)

In [21]:
prep=sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/precipitation-readings.csv")

L=prep.map(lambda line: line.split(";"))

st = L.map(lambda x: ( (x[0],x[1]) ,float(x[3])) )

max_st= st.reduceByKey(lambda x,y :x if x>=y else y)

max_st=max_st.filter(lambda x: x[1]>=10. and x[1]<=20.)

wst=max_st.map(lambda x: (x[0],float(x[1]) ))

wst.take(10)

[(('105220', '1997-07-01'), 18.7),
 (('105220', '2004-02-04'), 10.0),
 (('106570', '1997-06-10'), 12.8),
 (('106570', '2004-07-19'), 15.5),
 (('108320', '2010-06-30'), 10.0),
 (('114140', '2009-07-30'), 10.7),
 (('114140', '2012-08-26'), 11.5),
 (('114410', '2013-07-31'), 12.2),
 (('122260', '2010-08-03'), 10.1),
 (('123060', '1996-06-17'), 11.9)]

In [80]:
#st.take(10)

In [82]:
#station_temperature.join(st).collect()

In [22]:
max_temp_st.join(wst).collect()

[]

### Question 5

In [23]:
stations=sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/stations-Ostergotland.csv")
precipitation=sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/precipitation-readings.csv")

stations=stations.map(lambda line: line.split(";"))
precipitation=precipitation.map(lambda line:line.split(";"))

In [24]:
RDD = precipitation.map(lambda x: ( (x[1][0:4],x[1][5:7],x[1][8:10],x[0]),float(x[3]) ))

RDD = RDD.filter(lambda x: int(x[0][0]) >= 1993 and int(x[0][0]) <= 2016)

#############################

#RDD_filtered=RDD.map(lambda x: (x[0][0],x[0][1], x[0][2],x[0][3],float(x[1]) ))

RDD.take(10)

[(('1995', '08', '01', '103100'), 0.0),
 (('1995', '08', '01', '103100'), 0.0),
 (('1995', '08', '01', '103100'), 0.0),
 (('1995', '08', '01', '103100'), 0.0),
 (('1995', '08', '01', '103100'), 0.0),
 (('1995', '08', '01', '103100'), 0.0),
 (('1995', '08', '01', '103100'), 0.0),
 (('1995', '08', '01', '103100'), 0.0),
 (('1995', '08', '01', '103100'), 0.0),
 (('1995', '08', '01', '103100'), 0.0)]

In [25]:
mRDD = (RDD
           .mapValues(lambda x: (x, 1))
           .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))
           .mapValues(lambda x: x[0]/x[1]))
mRDD.sortByKey(ascending=False).take(10)

[(('2016', '07', '01', '99280'), 0.014285714285714287),
 (('2016', '07', '01', '98490'), 0.09999999999999999),
 (('2016', '07', '01', '98040'), 0.0),
 (('2016', '07', '01', '97530'), 0.62),
 (('2016', '07', '01', '97510'), 0.26),
 (('2016', '07', '01', '97370'), 0.02),
 (('2016', '07', '01', '97100'), 0.34285714285714286),
 (('2016', '07', '01', '96560'), 0.24285714285714288),
 (('2016', '07', '01', '96190'), 0.0),
 (('2016', '07', '01', '95540'), 0.0)]

In [30]:
M=mRDD.map(lambda x: ( (x[0][0],x[0][1],x[0][3]),x[1]) )\
.mapValues(lambda x: (x, 1))\
.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))\
.mapValues(lambda x: x[0]/x[1]).sortByKey()
M.take(10)

[(('1993', '01', '154860'), 0.1295698924731183),
 (('1993', '01', '188790'), 0.017633333333333338),
 (('1993', '01', '97510'), 0.030277777777777775),
 (('1993', '02', '154860'), 0.033384387351778656),
 (('1993', '02', '188790'), 0.036128364389233954),
 (('1993', '02', '97510'), 0.02008281573498965),
 (('1993', '03', '154860'), 0.028341013824884784),
 (('1993', '03', '188790'), 0.07755921770297648),
 (('1993', '03', '97510'), 0.026113671274961593),
 (('1993', '04', '154860'), 0.006488957902001381)]

In [29]:
stations_ost=sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/stations-Ostergotland.csv")

stations_ost=stations_ost.map(lambda line: line.split(";"))

stations_ost.map(lambda x : x[0])
stations_

['84260',
 '85630',
 '85460',
 '85450',
 '85490',
 '85650',
 '85390',
 '85410',
 '86470',
 '86440',
 '86420',
 '86340',
 '86350',
 '86360',
 '86370',
 '86200',
 '86330',
 '87140',
 '87150',
 '86130',
 '86090',
 '85280',
 '85250',
 '85240',
 '85270',
 '85220',
 '85210',
 '85180',
 '85040',
 '85050',
 '75520',
 '85160',
 '85600',
 '85130']

In [35]:

M.filter(lambda x : stations_ost in  x[0][2] ).take(10)

Traceback (most recent call last):
  File "C:\Users\quartermaine\Anaconda2\envs\python3\lib\site-packages\pyspark\cloudpickle.py", line 235, in dump
    return Pickler.dump(self, obj)
  File "C:\Users\quartermaine\Anaconda2\envs\python3\lib\pickle.py", line 409, in dump
    self.save(obj)
  File "C:\Users\quartermaine\Anaconda2\envs\python3\lib\pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "C:\Users\quartermaine\Anaconda2\envs\python3\lib\pickle.py", line 751, in save_tuple
    save(element)
  File "C:\Users\quartermaine\Anaconda2\envs\python3\lib\pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "C:\Users\quartermaine\Anaconda2\envs\python3\lib\site-packages\pyspark\cloudpickle.py", line 378, in save_function
    self.save_function_tuple(obj)
  File "C:\Users\quartermaine\Anaconda2\envs\python3\lib\site-packages\pyspark\cloudpickle.py", line 529, in save_function_tuple
    save(closure_values

PicklingError: Could not serialize object: Exception: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.